In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import Binarizer
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as matplot
import numpy as np

%matplotlib inline

## Import South East Brazil Weather Data

In [2]:
# Initially load all the data into the data frame
# Initially we will create a model so that we are able to predict the current temperature. 
# For that reason I am taking the temperature variable in a separate series which can act as a label later.
df = pd.read_csv('sudeste.csv')
temp = df.pop("temp")

In [3]:
df.head()

,wsid,wsnm,elvt,lat,lon,inme,city,prov,mdct,date,...,tmax,dmax,tmin,dmin,hmdy,hmax,hmin,wdsp,wdct,gust
0,178,SÃO GONÇALO,237.0,-6.835777,-38.311583,A333,São Gonçalo,RJ,2007-11-06 00:00:00,2007-11-06,...,29.7,16.8,25.5,10.8,35.0,58.0,32.0,3.2,101.0,6.5
1,178,SÃO GONÇALO,237.0,-6.835777,-38.311583,A333,São Gonçalo,RJ,2007-11-06 01:00:00,2007-11-06,...,29.9,13.6,29.0,12.2,39.0,39.0,35.0,3.6,94.0,6.4
2,178,SÃO GONÇALO,237.0,-6.835777,-38.311583,A333,São Gonçalo,RJ,2007-11-06 02:00:00,2007-11-06,...,29.0,14.0,27.4,13.6,44.0,44.0,39.0,2.5,93.0,6.9
3,178,SÃO GONÇALO,237.0,-6.835777,-38.311583,A333,São Gonçalo,RJ,2007-11-06 03:00:00,2007-11-06,...,27.4,16.9,25.8,14.1,58.0,58.0,44.0,1.7,96.0,5.8
4,178,SÃO GONÇALO,237.0,-6.835777,-38.311583,A333,São Gonçalo,RJ,2007-11-06 04:00:00,2007-11-06,...,26.3,17.0,25.3,16.4,57.0,58.0,56.0,3.1,110.0,7.5


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9779168 entries, 0 to 9779167
Data columns (total 30 columns):
wsid    int64
wsnm    object
elvt    float64
lat     float64
lon     float64
inme    object
city    object
prov    object
mdct    object
date    object
yr      int64
mo      int64
da      int64
hr      int64
prcp    float64
stp     float64
smax    float64
smin    float64
gbrd    float64
dewp    float64
tmax    float64
dmax    float64
tmin    float64
dmin    float64
hmdy    float64
hmax    float64
hmin    float64
wdsp    float64
wdct    float64
gust    float64
dtypes: float64(19), int64(5), object(6)
memory usage: 2.2+ GB


In [5]:
# To find the statistical information of the data
df.describe()

,wsid,elvt,lat,lon,yr,mo,da,hr,prcp,stp,...,tmax,dmax,tmin,dmin,hmdy,hmax,hmin,wdsp,wdct,gust
count,9.779168e+06,9.779168e+06,9.779168e+06,9.779168e+06,9.779168e+06,9.779168e+06,9.779168e+06,9.779168e+06,1.407984e+06,9.779168e+06,...,9.779142e+06,9.778858e+06,9.779134e+06,9.778361e+06,9.779168e+06,9.779156e+06,9.779124e+06,8.853607e+06,9.779168e+06,9.462694e+06
mean,3.592531e+02,5.940923e+02,-2.023082e+01,-4.464590e+01,2.011164e+03,6.521960e+00,1.575465e+01,1.150000e+01,9.366544e-01,8.804292e+02,...,2.110503e+01,1.524025e+01,1.986418e+01,1.422089e+01,6.726667e+01,6.996991e+01,6.441965e+01,1.998156e+00,1.385991e+02,4.494015e+00
std,3.901630e+01,3.980379e+02,3.172643e+00,4.882117e+00,3.207774e+00,3.425538e+00,8.802154e+00,6.922190e+00,2.923291e+00,2.482650e+02,...,7.545549e+00,5.866811e+00,7.134849e+00,5.777089e+00,2.654213e+01,2.643371e+01,2.656550e+01,1.618531e+00,1.052018e+02,2.981790e+00
min,1.780000e+02,0.000000e+00,-2.496282e+01,-5.667732e+01,2.000000e+03,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,-3.200000e+00,-1.000000e+01,-8.500000e+00,-1.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,3.280000e+02,2.830000e+02,-2.239580e+01,-4.743410e+01,2.009000e+03,4.000000e+00,8.000000e+00,5.000000e+00,0.000000e+00,9.113000e+02,...,1.820000e+01,1.270000e+01,1.720000e+01,1.160000e+01,5.300000e+01,5.800000e+01,4.900000e+01,8.000000e-01,5.600000e+01,2.300000e+00
50%,3.580000e+02,5.730000e+02,-2.075041e+01,-4.445378e+01,2.011000e+03,7.000000e+00,1.600000e+01,1.200000e+01,0.000000e+00,9.442000e+02,...,2.190000e+01,1.660000e+01,2.080000e+01,1.560000e+01,7.400000e+01,7.800000e+01,7.000000e+01,1.700000e+00,1.140000e+02,4.200000e+00
75%,3.940000e+02,8.750000e+02,-1.891707e+01,-4.243575e+01,2.014000e+03,9.000000e+00,2.300000e+01,1.800000e+01,6.000000e-01,9.731000e+02,...,2.580000e+01,1.940000e+01,2.420000e+01,1.840000e+01,8.900000e+01,9.100000e+01,8.600000e+01,2.900000e+00,2.160000e+02,6.300000e+00
max,4.230000e+02,1.758000e+03,0.000000e+00,0.000000e+00,2.016000e+03,1.200000e+01,3.100000e+01,2.300000e+01,1.000000e+02,1.050000e+03,...,4.500000e+01,4.480000e+01,4.500000e+01,4.490000e+01,1.000000e+02,1.000000e+02,1.000000e+02,1.980000e+01,3.600000e+02,5.000000e+01


In [6]:
df.shape

(9779168, 30)

In [7]:
# In the data it has been mentioned that all the weather stations started operating from the year 2000 so we can drop all the 
# rows which has the data from previous years and will not be relevant to current weather data.
df[df.yr < 2000]
df.shape

(9779168, 30)

In [8]:
# Here we can see that all the data in the column year is more than 2000 so no rows are dropped based on that condition
# But we will verify it again by looking at the minimum value of the year column
print (df['yr'].min())
print (df['yr'].max())

2000
2016


Therefore we have verified that the minimum value of the data in the column yr is 2000 and that is when all the weather stations started operating. Now we have eliminated one condition of cleaning the data i.e. all the weather stations are running from the 
year 2000 till 2016.

In [9]:
df.dropna(axis=0, how='all')
df.shape

(9779168, 30)

Here we are checking if there is any row where all the values are NA so that we can drop it. Becasue that particular row is of no use and can be easily dropped. From the results we find that there is no row where all the rows are NA values.

In [10]:
df.dropna(axis=1, how='all')
df.shape

(9779168, 30)

Here we are checking if there is any column where all the values are NA so that we can drop it. From the results it is clear that there is no column with all NA values.

In [11]:
# Check for any duplicate data
df.duplicated().value_counts()

False    9779168
dtype: int64

All the rows are unique and there are no duplicate rows which could be dropped.

In [12]:
df.count()

wsid    9779168
wsnm    9779168
elvt    9779168
lat     9779168
lon     9779168
inme    9779168
city    9779168
prov    9779168
mdct    9779168
date    9779168
yr      9779168
mo      9779168
da      9779168
hr      9779168
prcp    1407984
stp     9779168
smax    9779168
smin    9779168
gbrd    5670348
dewp    9778693
tmax    9779142
dmax    9778858
tmin    9779134
dmin    9778361
hmdy    9779168
hmax    9779156
hmin    9779124
wdsp    8853607
wdct    9779168
gust    9462694
dtype: int64

With df.count() we are able to see how many entries are present in each column. By which we can get to know how many missin values are present in each column. From the above table it is clearly evident that most of the columns dont have any missing values. But the column Precipitation has very few values in there as compared to the rest of the columns. The column of solar radiation also has some data missing and some other few columns with some data missing.
Now we will have to decide which column data needs to be imputed with some other data or what needs to be done about it.

In [13]:
# Impute missing values with median values for the columns with some missing data
missing_columns = ["gbrd", "dewp", "tmax", "dmax", "tmin", "dmin", "hmax", "hmin", "wdsp", "gust"]

for missing in missing_columns:
    df[missing].fillna(df[missing].median(), inplace=True)

In [14]:
df.count()

wsid    9779168
wsnm    9779168
elvt    9779168
lat     9779168
lon     9779168
inme    9779168
city    9779168
prov    9779168
mdct    9779168
date    9779168
yr      9779168
mo      9779168
da      9779168
hr      9779168
prcp    1407984
stp     9779168
smax    9779168
smin    9779168
gbrd    9779168
dewp    9779168
tmax    9779168
dmax    9779168
tmin    9779168
dmin    9779168
hmdy    9779168
hmax    9779168
hmin    9779168
wdsp    9779168
wdct    9779168
gust    9779168
dtype: int64

Now the only column which is not required here is the precipitation column which can be dropped. We will test our model and the results and based on that we can decide whether to include the parameter or not.

In [15]:
df.drop(['prcp'], axis=1, inplace=True)
df.count()

wsid    9779168
wsnm    9779168
elvt    9779168
lat     9779168
lon     9779168
inme    9779168
city    9779168
prov    9779168
mdct    9779168
date    9779168
yr      9779168
mo      9779168
da      9779168
hr      9779168
stp     9779168
smax    9779168
smin    9779168
gbrd    9779168
dewp    9779168
tmax    9779168
dmax    9779168
tmin    9779168
dmin    9779168
hmdy    9779168
hmax    9779168
hmin    9779168
wdsp    9779168
wdct    9779168
gust    9779168
dtype: int64

Now the data is uniform and all the columns are filled with exact number of rows. We have dropped the column of Precipitation but we can add it later if required and the results are not in our favour. Now, lets focus on the main task of predicting the Temperature (current temperature) using the above data.

### Split the data into train and test data

In [41]:
# Import the libraries to create the training and test data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df, temp, test_size=0.20, random_state=42)

## Train on Numeric Features

In [42]:
# Taking all the numeric features in a list
#numeric_data = ['elvt', 'lat', 'lon', 'stp', 'smax', 'smin', 'gbrd', 'dewp', 'tmax', 'dmax', 'tmin', 'dmin', 'hmdy', 'hmax',
#               'hmin', 'wdsp', 'wdct', 'gust']
    
numeric_data = ['stp', 'smax', 'smin', 'gbrd', 'dewp', 'hmdy',
                'wdsp', 'wdct', 'gust']
X_numeric = X_train[numeric_data]
X_numeric.head()

,stp,smax,smin,gbrd,dewp,hmdy,wdsp,wdct,gust
8240730,970.5,970.7,970.3,8.923,17.2,97.0,0.7,341.0,1.7
1903388,913.7,913.7,913.2,232.000,14.5,78.0,3.4,124.0,8.5
6378219,823.1,823.4,822.9,366.207,9.7,82.0,7.2,318.0,14.5
7239226,942.5,943.4,942.4,863.427,17.7,62.0,1.3,101.0,5.3
5311471,881.7,881.7,881.3,863.427,6.7,40.0,1.7,114.0,1.8


In [43]:
X_numeric.count()

stp     7823334
smax    7823334
smin    7823334
gbrd    7823334
dewp    7823334
hmdy    7823334
wdsp    7823334
wdct    7823334
gust    7823334
dtype: int64

Since there are no missing values here we don't need to fill any columns with any median values.

In [48]:
X_numeric = X_numeric.fillna(X_numeric.mean())

In [49]:
X_numeric = X_numeric.reset_index()

## Random Forest Baseline Model

In [52]:
# Create the baseline model using Random Forest
model_1 = RandomForestRegressor(oob_score=True, n_jobs = -1, random_state=42)

## Out of Bag Score (Built in Cross Validation)

In [53]:
# Fit the Random Forest Model and evaluate the OOB score
model_1.fit(X_numeric, y_train)

# Calculate the OOB score
print ("The OOB score is: " + str(model_1.oob_score_))

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').